In [1]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



In [4]:
ls ${FD_WRK}/annotation_fragment

Input   Input2  Input4  TFX2_DMSO  TFX3_DMSO  TFX4_DMSO  TFX5_DMSO  TFX_DMSO
Input1  Input3  Input5  TFX2_Dex   TFX3_Dex   TFX4_Dex   TFX5_Dex   TFX_Dex


In [6]:
ls ${FD_WRK}/annotation_fragment/Input1/chr*

/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr1.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr10.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr11.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr12.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr13.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr14.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr15.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr16.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr17.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr17.bed.gz
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr18.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr19.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr2.bed
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr20.bed
/work/kk319/out/Com

In [7]:
head ${FD_WRK}/annotation_fragment/Input1/chr17.bed

chr17	201123	201925	1	chr17	201116	201126	NFKB/1	9.0328	3
chr17	201123	201925	1	chr17	201136	201152	MZF1	9.0967	16
chr17	201123	201925	1	chr17	201138	201148	NFKB/1	8.1413	10
chr17	201123	201925	1	chr17	201141	201154	SPDEF/1	4.4219	13
chr17	201123	201925	1	chr17	201145	201160	CENBP	9.3664	15
chr17	201123	201925	1	chr17	201150	201171	GRHL	7.965733333	21
chr17	201123	201925	1	chr17	201153	201162	CCAAT/CEBP	7.8645	9
chr17	201123	201925	1	chr17	201168	201179	BCL6/2	8.3878	11
chr17	201123	201925	1	chr17	201171	201183	NFAC/2	7.809	12
chr17	201123	201925	1	chr17	201173	201182	NFAT/1	9.5126	9


In [ ]:
CHROM=chr17
FD_BEDS=($(ls -d ${FD_WRK}/annotation_fragment/*/))

In [ ]:
awk -F $'\t' '($2 >= 8148000 && $2 <= 8160000)'

In [9]:
MOTIF=NFKB/1
FNAME=$(echo ${MOTIF} | sed -e 's/\//_/g')
echo ${MOTIF}
echo ${FNAME}
head ${FD_WRK}/annotation_fragment/Input1/chr17.bed |\
    awk -F $'\t' -v MOTIF=${MOTIF} '($8 == MOTIF)'

NFKB/1
NFKB_1
chr17	201123	201925	1	chr17	201116	201126	NFKB/1	9.0328	3
chr17	201123	201925	1	chr17	201138	201148	NFKB/1	8.1413	10


In [10]:
source ../config_duke.sh
source ../load_module_bedtools.sh

sbatch -p scavenger \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/filter_annotation_fragment_chrom_NR20.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
MOTIF=NR/20
FNAME=$(echo ${MOTIF} | sed -e 's/\//_/g')

### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/annotation_fragment/*/))
FN_BED=${CHROM}.bed

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    #SAMPLE=$(basename ${FD_BED})
    #FD_OUT=${FD_WRK}/coverage/${SAMPLE}
    FD_OUT=${FD_BED}
    FN_OUT=${FNAME}_${CHROM}.txt
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    echo "Show the last few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    tail -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### filter out the specified motif and given chromosome for each sample
    cat ${FD_BED}/${FN_BED} | awk -F $'\t' -v MOTIF=${MOTIF} '($8 == MOTIF)' > ${FD_OUT}/${FN_OUT}
    
    ### END Message
    echo
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    head -n 3 ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the last few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    tail -n 3 ${FD_OUT}/${FN_OUT}
    echo
done
echo Done!
EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

You are on Duke Server: DCC
Bedtools 2.27.1
>Submitted batch job 15214030


In [12]:
readlink -f ${FD_LOG}/filter_annotation_fragment_chrom_NR20.23.txt

/work/kk319/out/CombEffect_STARR/log/filter_annotation_fragment_chrom_NR20.23.txt


In [1]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



In [2]:
FD_BED=${FD_WRK}/annotation_fragment/Input1

In [3]:
ls ${FD_BED}/NR_20*                                                                

/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr1.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr10.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr11.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr12.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr13.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr14.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr15.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr16.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr17.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr18.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr19.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr2.txt
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr20.txt
/work/kk319/ou

In [4]:
ls -lh ${FD_BED}/NR_20*  

-rw-r--r--. 1 kk319 root  40M Jul 21 18:06 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr1.txt
-rw-r--r--. 1 kk319 root  25M Jul 21 17:27 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr10.txt
-rw-r--r--. 1 kk319 root  25M Jul 21 17:00 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr11.txt
-rw-r--r--. 1 kk319 root  22M Jul 21 17:00 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr12.txt
-rw-r--r--. 1 kk319 root  14M Jul 21 16:50 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr13.txt
-rw-r--r--. 1 kk319 root  15M Jul 21 16:48 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr14.txt
-rw-r--r--. 1 kk319 root  14M Jul 21 16:52 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr15.txt
-rw-r--r--. 1 kk319 root  16M Jul 21 16:49 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20_chr16.txt
-rw-r--r--. 1 kk319 root  15M Jul 21 16:50 /work/